In [ ]:
pip install cmake

In [ ]:
pip install --upgrade pip setuptools wheel


In [ ]:
pip install dlib

In [ ]:
pip install face_recognition

In [ ]:
pip install openpyxl

In [ ]:
#1
#cpaturing photos
# Import necessary libraries
import cv2
import os

# Ask the user to enter their name
user_name = input("Enter your name: ").strip()

# Initialize the camera
cap = cv2.VideoCapture(0)

# Create a directory for storing the dataset
if not os.path.exists("dataset"):
    os.makedirs("dataset")

# Create a dataset
count = 0
max_photos = 50 # Set the number of photos to capture
while count < max_photos:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image. Exiting...")
        break
    cv2.imshow("Capturing Photos", frame)
    
    # Save the current frame as an image
    count += 1
    filename = f"dataset/{user_name}_{count}.jpg"  # Save photo with number
    cv2.imwrite(filename, frame)
    print(f"Photo {count}/{max_photos} saved as {filename}")
    
    # Wait for a short duration before capturing the next photo
    cv2.waitKey(100)  # 100ms delay between captures

# Release the camera and close the window
cap.release()
cv2.destroyAllWindows()
print(f"Captured {count} photos for {user_name}.")



Photo 1/50 saved as dataset/Mohana_1.jpg
Photo 2/50 saved as dataset/Mohana_2.jpg
Photo 3/50 saved as dataset/Mohana_3.jpg
Photo 4/50 saved as dataset/Mohana_4.jpg
Photo 5/50 saved as dataset/Mohana_5.jpg
Photo 6/50 saved as dataset/Mohana_6.jpg
Photo 7/50 saved as dataset/Mohana_7.jpg
Photo 8/50 saved as dataset/Mohana_8.jpg
Photo 9/50 saved as dataset/Mohana_9.jpg
Photo 10/50 saved as dataset/Mohana_10.jpg
Photo 11/50 saved as dataset/Mohana_11.jpg
Photo 12/50 saved as dataset/Mohana_12.jpg
Photo 13/50 saved as dataset/Mohana_13.jpg
Photo 14/50 saved as dataset/Mohana_14.jpg
Photo 15/50 saved as dataset/Mohana_15.jpg
Photo 16/50 saved as dataset/Mohana_16.jpg
Photo 17/50 saved as dataset/Mohana_17.jpg
Photo 18/50 saved as dataset/Mohana_18.jpg
Photo 19/50 saved as dataset/Mohana_19.jpg
Photo 20/50 saved as dataset/Mohana_20.jpg
Photo 21/50 saved as dataset/Mohana_21.jpg
Photo 22/50 saved as dataset/Mohana_22.jpg
Photo 23/50 saved as dataset/Mohana_23.jpg
Photo 24/50 saved as dataset/

In [4]:
#2
#training data
# Import necessary libraries
import face_recognition
import os
import numpy as np
import json

# Create a directory for storing the embeddings
if not os.path.exists("embeddings"):
    os.makedirs("embeddings")

# Load the images and extract names
image_paths = [os.path.join("dataset", f) for f in os.listdir("dataset")]
embeddings = []
names = []
for image_path in image_paths:
    # Extract the name from the file path (ignoring numbering)
    name = os.path.basename(image_path).split('_')[0]
    image = face_recognition.load_image_file(image_path)
    face_locations = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(image, face_locations)
    if len(face_encodings) == 1:
        embeddings.append(face_encodings[0])
        names.append(name)

# Save the embeddings and names
np.save("embeddings/embeddings.npy", embeddings)
with open("embeddings/names.json", "w") as f:
    json.dump(names, f)



In [12]:
#3
#recognition and attendance marking
import cv2
import json
import face_recognition
import numpy as np
from datetime import datetime
import os

# Load the embeddings from .npy file
embeddings = np.load("embeddings/embeddings.npy")

# Initialize the camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Ensure the attendance file exists with valid JSON content
if not os.path.exists("attendance.json"):
    with open("attendance.json", "w") as f:
        json.dump([], f)  # Write an empty list

# Load the attendance data
try:
    with open("attendance.json", "r") as f:
        attendance = json.load(f)
        if not isinstance(attendance, list):  # Ensure it’s a list
            attendance = []
except (FileNotFoundError, json.JSONDecodeError):
    attendance = []  # Create an empty list if no file exists or it's empty

# Load known names from names.json
with open("embeddings/names.json", "r") as f:
    names = json.load(f)

# Create a set to track attendance for the current session
marked_in_session = set()

# Main loop to process the video frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Find all face locations and encodings in the current frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Compare faces with known embeddings
        matches = face_recognition.compare_faces(embeddings, face_encoding)
        face_distances = face_recognition.face_distance(embeddings, face_encoding)
        best_match_index = np.argmin(face_distances) if face_distances.size > 0 else -1

        if best_match_index != -1 and matches[best_match_index]:
            name = names[best_match_index]  # Retrieve username

            # Mark attendance if not already marked in this session
            if name not in marked_in_session:
                attendance.append({
                    "name": name,
                    "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })
                marked_in_session.add(name)  # Add to the session set
                print(f"Attendance marked for {name}")

            # Draw a rectangle around the face and put the name
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        else:
            # Mark the face as "Unknown" if no match is found
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.putText(frame, "Unknown", (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Show the frame with bounding boxes and names
    cv2.imshow("Frame", frame)

    # Save attendance to the JSON file after each recognition
    with open("attendance.json", "w") as f:
        json.dump(attendance, f, indent=4)

    # Exit on pressing 'q'
    if cv2.waitKey(1) == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()
  


Attendance marked for Mohana


In [15]:
#4
#writing attendance in excel
import pandas as pd
import json

# Load the attendance log from the JSON file
with open('attendance.json', 'r') as f:
    attendance_log = json.load(f)
attendance_list=[{"User":i['name'],"Timestamp":i['time']} for i in  attendance_log ]
# Convert the attendance log to a Pandas dataframe
df = pd.DataFrame(attendance_list)

# Write the attendance data to an Excel file
df.to_excel('attendance_log.xlsx', index=False, engine='openpyxl')